In [1]:
#importing necessary python packages
import sksurv
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel

In [2]:
# Importing the dataset
dt = pd.read_excel('Flux Clinical.xlsx',index_col=0)

In [3]:
del dt['Column1']
del dt['Patient\'s Vital Status_Dead']
del dt['Patient\'s Vital Status_Alive']

In [4]:
c = pd.read_excel('cox Luad.xlsx')

In [8]:
result=c.to_records(index=False)

In [9]:
c['Patient\'s Vital Status' ]=c['Patient\'s Vital Status' ].astype('bool')

In [10]:
result

rec.array([( True, 0.0000e+00), ( True, 5.0030e+01), (False, 3.9800e+00),
           ( True, 1.9940e+01), ( True, 1.3990e+01), ( True, 4.4970e+01),
           ( True, 3.6990e+01), (False, 0.0000e+00), (False, 9.9500e+00),
           (False, 2.4010e+01), ( True, 4.7010e+01), (False, 8.0200e+00),
           ( True, 1.8990e+01), ( True, 2.0040e+01), ( True, 0.0000e+00),
           (False, 2.9900e+00), ( True, 1.4950e+01), (False, 9.0000e+00),
           ( True, 2.9960e+01), ( True, 1.1990e+01), ( True, 2.9990e+01),
           ( True, 2.1980e+01), ( True, 2.5990e+01), ( True, 2.5990e+01),
           ( True, 2.5000e+01), ( True, 2.5000e+01), ( True, 2.3980e+01),
           (False, 1.5010e+01), ( True, 1.5010e+01), ( True, 4.9600e+00),
           ( True, 2.8980e+01), ( True, 2.2010e+01), (False, 9.0300e+00),
           ( True, 2.0400e+00), ( True, 4.6000e-01), ( True, 7.3900e+00),
           ( True, 5.9800e+00), ( True, 8.6700e+00), ( True, 9.7670e+01),
           ( True, 1.2070e+02), (False

In [11]:
y = result

In [12]:
dt.shape

(517, 2306)

In [13]:
dt.head(2)

,'3DSPHR','3HBCOAHLm','ABTArm','ABTD','ACACT1r','ACACT1rm','ACACT1x','ACOAD9m','ACOAO7p','ACOATA',...,Tissue Source Site_MN,Tissue Source Site_MP,Tissue Source Site_NJ,Tissue Source Site_O1,Tissue Source Site_S2,Person Neoplasm Status_TUMOR FREE,Person Neoplasm Status_WITH TUMOR,Vial number_A,Vial number_B,Overall Survival (Months)
HUG0_SYMBOL,,,,,,,,,,,,,,,,,,,,,
TCGA-05-4244-01,11.448262,154.187481,-583.764387,-14.237357,125.403072,235.338244,457.654536,74.716367,72.493260,-165.591744,...,0,0,0,0,0,1,0,1,0,0.00
TCGA-05-4249-01,9.173955,162.200877,-442.863722,-5.338254,128.134267,137.492687,149.939274,75.026103,56.620656,-73.252506,...,0,0,0,0,0,1,0,1,0,50.03


In [14]:
X = dt.iloc[:,0:2305]
y1 = dt.iloc[:, -1].values

In [15]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y1_train, y1_test = train_test_split(X, y1, test_size = 0.2, random_state = 0)
X_train, X_val, y1_train, y1_val = train_test_split(X_train, y1_train, test_size=0.2, random_state=0)

In [16]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [17]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
X_val = sc_X.transform(X_val)

/Users/Zayeem/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/Zayeem/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Users/Zayeem/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """
/Users/Zayeem/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


In [18]:
# Create a random forest Regressor
rgr = RandomForestRegressor(n_estimators=100, random_state=0)

In [19]:
# Train the Regressor
rgr.fit(X_train, y1_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [ ]:
# Print the name and importance of each feature
for feature in zip(X.columns, rgr.feature_importances_):
    print(feature)

In [21]:
#Select the best features from the regressor
sfm = SelectFromModel(rgr)

In [22]:
# Train the Regressor
sfm.fit(X_train, y1_train)

SelectFromModel(estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False),
        max_features=None, norm_order=1, prefit=False, threshold=None)

In [23]:
# Print the names of the most important features
for feature_list_index in sfm.get_support(indices=True):
    print(X.columns[feature_list_index])

'ACACT1rm'
'ACOAD9m'
'ACONT'
'ACONTm'
'ADK1'
'ADK1m'
'ADK3'
'ADK3m'
'AKGDm'
'ALATA_L'
'ARTPLM3'
'ASPTA'
'CATp'
'CKc'
'CSNATp'
'CSNATr'
'CYSGLTH'
'CYTK10'
'CYTK13'
'CYTK2'
'CYTK3'
'CYTK6'
'CYTK7'
'DURAD2'
'FACOAL1821'
'FAS120COA'
'FAS160COA'
'GALT'
'GAPD'
'GGT6'
'GLPASE1'
'GLUDxm'
'GLUDym'
'GRTTx'
'H2CO3Dm'
'HMGCOASi'
'HMGCOASim'
'ITCOALm'
'LDH_Lm'
'LPS2e'
'MCD'
'MDH'
'MDHm'
'MMMm'
'MTHFC'
'MTHFD'
'MTHFD2'
'NDPK1'
'NDPK2'
'NDPK7'
'NDPK9'
'PEPCKm'
'PETOHMr_hs'
'PGI'
'PUNP3'
'PYNP2r'
'RNDR1'
'RNDR4'
'SBTR'
'TALA'
'TMDK1m'
'TMDPP'
'UGLT'
'UMPK'
'UMPK3'
'UMPK4'
'UMPK5'
'VALTA'
'r0021'
'r0081'
'r0086'
'r0173'
'r0193'
'r0226'
'r0245'
'r0321'
'r0330'
'r0384'
'r0392'
'r0407'
'r0408'
'r0413'
'r0438'
'r0475'
'r0483'
'r0556'
'r0620'
'r0627'
'r0643'
'r0792'
'r1156'
'r1384'
'RE0452M'
'RE0453C'
'RE0453M'
'RE0579C'
'RE0926E'
'RE0935E'
'RE1530C'
'BAAT5x'
'RE3273C'
'RE3301C'
'RE3347C'
'DTMPKm'
'MTHFR3'
'10FTHFtm'
'2HBt2'
'ACALDt'
'ACt2r'
'ADEt'
'ALADGLYexR'
'ARGt4'
'ARGtiDF'
'ASCBt'
'ASNt4'
'ASNtN1'
'AT

In [24]:
sfm.get_support().sum()

382

In [25]:
# Transform the data to create a new dataset containing only the most important features
# Note: We have to apply the transform to both the training X and test X data.
X_important_train = sfm.transform(X_train)
X_important_val = sfm.transform(X_val)
X_important_test = sfm.transform(X_test)

In [26]:
from sksurv.linear_model import CoxPHSurvivalAnalysis
estimator = CoxPHSurvivalAnalysis(verbose=1,alpha=1)
estimator.fit(X_important_train, y_train)

iter     11: optimization converged


CoxPHSurvivalAnalysis(alpha=1, n_iter=100, tol=1e-09, verbose=1)

In [27]:
from sksurv.metrics import concordance_index_censored

prediction = estimator.predict(X_important_val)
result = concordance_index_censored(y_val['Patient\'s Vital Status'], y_val['Survival '], prediction)
result[0]

0.6476683937823834

In [28]:
# KFOlD
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
kfold = KFold(n_splits=3, random_state=0)
results = cross_val_score(estimator, X_important_train, y_train, cv=kfold)
print(results.mean(), results.std())

iter     15: optimization converged
iter     14: optimization converged
iter     13: optimization converged
0.7044007001761329 0.006218379957709117


In [ ]:
-------------------------------OPTIMISATION---------------------------------------

In [29]:
from sklearn.model_selection import RandomizedSearchCV
param_grid = {  'alpha':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,2,3,4,5,6,7,8,9,10],
                'n_iter':[1,2,3,4,5,6,7,8,9,10,100],
                'tol':[0.001,0.01,0.1,0.005,0.05,0.5,1.0,2,5,10,15,20,25,30],
                
                 }  
regressor = CoxPHSurvivalAnalysis()
random=RandomizedSearchCV(estimator=regressor,param_distributions=param_grid,cv=3,n_iter=10)
random_result=random.fit(X_important_train,y_train)
print("Best: %f using %s" % (random_result.best_score_,random_result.best_params_))

/Users/Zayeem/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:528: ConvergenceWarning: Optimization did not converge: Maximum number of iterations has been exceeded.
  estimator.fit(X_train, y_train, **fit_params)
/Users/Zayeem/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:528: ConvergenceWarning: Optimization did not converge: Maximum number of iterations has been exceeded.
  estimator.fit(X_train, y_train, **fit_params)
/Users/Zayeem/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:528: ConvergenceWarning: Optimization did not converge: Maximum number of iterations has been exceeded.
  estimator.fit(X_train, y_train, **fit_params)
/Users/Zayeem/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:528: ConvergenceWarning: Optimization did not converge: Maximum number of iterations has been exceeded.
  estimator.fit(X_train, y_train, **fit_params)
/Users/Zayeem/anaconda3/lib/

Best: 0.726147 using {'tol': 0.005, 'n_iter': 4, 'alpha': 10}


In [30]:
from sksurv.linear_model import CoxPHSurvivalAnalysis
estimator = CoxPHSurvivalAnalysis(alpha=10,n_iter=4,tol=0.005,verbose=0)
estimator.fit(X_important_train, y_train)

CoxPHSurvivalAnalysis(alpha=10, n_iter=4, tol=0.005, verbose=0)

In [31]:
from sksurv.metrics import concordance_index_censored

prediction = estimator.predict(X_important_val)
result = concordance_index_censored(y_val['Patient\'s Vital Status'], y_val['Survival '], prediction)
result[0]

0.6545768566493955

In [32]:
#################Testing Set###########################
from sksurv.linear_model import CoxPHSurvivalAnalysis
estimator = CoxPHSurvivalAnalysis(alpha=10,n_iter=4,tol=0.005,verbose=0)
estimator.fit(X_important_train, y_train)

CoxPHSurvivalAnalysis(alpha=10, n_iter=4, tol=0.005, verbose=0)

In [33]:
from sksurv.metrics import concordance_index_censored

prediction = estimator.predict(X_important_test)
result = concordance_index_censored(y_test['Patient\'s Vital Status'], y_test['Survival '], prediction)
result[0]

0.7550479973518702